<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Langchain_weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 9.8 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19580 sha256=9d215c2193338cae027078d998f0410b941005eb3c0f78357dac5cfd00ce6d6d
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is

In [18]:
!pip install -qU openai langchain tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.1 MB/s eta 0:00:00


In [5]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 9 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,257 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120493 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [6]:
! apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1 [186 kB]
Fetched 186 kB in 0s (820 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120540 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.1_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [7]:
! pip install -U pytesseract

In [8]:
! pip install pdf2image

In [11]:
! pip install -qU PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00


## Set up required API keys

In [3]:
import os
import openai
from getpass import getpass
#
os.environ["OPENAI_API_KEY"] = getpass()
#
WEAVIATE_URL = getpass()
#
os.environ["WEAVIATE_API_KEY"] = getpass()

··········
··········
··········


## Inference Script

In [26]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.document_loaders import TextLoader
import openai
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import messages_from_dict, messages_to_dict

#
from langchain.llms import HuggingFaceHub
from getpass import getpass
import os
import json
import io
import pytesseract
import shutil
import random
from PIL import Image
from pdf2image import convert_from_path
#
from configparser import ConfigParser
#
ai_for_world_config = ConfigParser()
#
def read_config(parser: ConfigParser, location: str) -> None:
    assert parser.read(location), f"Could not read config {location}"
#
def is_scanned_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
      text += page.extract_text()

    # if no text found, it is likely a scanned pdf
    if text == "":
        print("Scanned PDF")
        return True

    else:
        print("Text-based PDF")
        return False
# Convert pdf to image format
def convert_pdf_to_image(pdf_path):
    pages = convert_from_path(pdf_path, 350)
    #
    output_dir="pdf_img_dir"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"
        image_path = os.path.join(output_dir,image_name)
        page.save(image_path, "JPEG")
        i = i+1
    return output_dir

# Extract text from image
#
def extract_text_from_image(image_path):
    extractedInformation = pytesseract.image_to_string(Image.open(image_path))
    return extractedInformation
#
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore(text_chunks,api_key):
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    # embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    #
    vectorstore= Weaviate.from_texts(texts=text_chunks,embedding=embeddings, weaviate_url=WEAVIATE_URL,by_text=False)
    #vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore,api_key):
    llm = ChatOpenAI(openai_api_key=api_key)
    # llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectorstore.as_retriever(), memory=memory
    )
    return conversation_chain

#
def preprocess_function(request_data,content_type=None):
    #
    print(f"request_data: {request_data}")
    #
    pdf_docs = [request_data["pdf1"]]
    #
    pdf2 = request_data.get("pdf2", None)
    pdf3 = request_data.get("pdf3", None)
    #
    if pdf2 and isinstance(pdf2, io.BytesIO) and pdf2.getbuffer().nbytes > 0:
        pdf_docs.append(pdf2)
    if pdf3 and isinstance(pdf3, io.BytesIO) and pdf3.getbuffer().nbytes > 0:
        pdf_docs.append(pdf3)
    #
    prompt = [request_data["question1"]]
    #Append if any more questions are provided
    q2 = request_data.get("question2", None)
    q3 = request_data.get("question3", None)
    print(f"q2: {q2}")
    print(f"q3: {q3}")
    #
    if q2:
        prompt.append(request_data["question2"])
    if q3:
        prompt.append(request_data["question3"])
    #
    text = ""
    #
    print(pdf_docs)
    print(prompt)
    i =0
    for pdf in pdf_docs:
        flag = is_scanned_pdf(pdf)
        pdf_name = f"pdf{i+1}"
        f_path = os.path.join("/tmp", pdf_name)
        print(f_path)
        if flag:
            with open(f_path, "wb") as f:
                f.write(pdf.getbuffer())
            print(f"filesize : {os.path.getsize(f_path)}")
            image_dir = convert_pdf_to_image(f_path)
            for file in os.listdir(image_dir):
                image_path = os.path.join(image_dir,file)
                text += extract_text_from_image(image_path)
            print(f"Text read from image: {text}")

        else:
          try:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
            print(f"Text read from pdf: {text}")
          except:
            print("Error reading pdf")

    return (text,prompt)
#
def model_load_function(model_path):
    '''
    CONFIG_FILE = os.path.join(model_path, "env.conf")
    read_config(ai_for_world_config, CONFIG_FILE)
    api_key = ai_for_world_config.get("openai", "api_key").strip()
    '''
    api_key = os.environ["OPENAI_API_KEY"]
    return api_key
#
def predict_function(context, api_key):
    raw_text,prompt = context
    print(prompt)
    # get the text chunks
    text_chunks = get_text_chunks(raw_text)

    # create vector store
    vectorstore = get_vectorstore(text_chunks,api_key)

    # create conversation chain
    conversation = get_conversation_chain(vectorstore,api_key)
    #
    res_out = []
    extracted_messages_mem = []
    for i in range(len(prompt)):
        response = conversation.run(prompt[i])
        res_out.append(response)
        extracted_messages_mem.append(conversation.memory.chat_memory.messages)
    return (res_out,extracted_messages_mem)
#

def postprocess_function(predictions, content_type=None):
    res_out,extracted_messages_mem = predictions
    output_dir="pdf_img_dir"
    if os.path.exists(output_dir):
        for  file in os.listdir(output_dir):
            file_name = os.path.join(output_dir,file)
            if os.path.exists(file_name) and os.path.isfile(file_name):
                print(f"File {file_name} exists. Hence Removing")
                os.remove(file_name)
    return json.dumps({"response": res_out})



In [13]:
_Krequest_data ={'pdf1':"/content/Git_Cheatsheet_KDnuggets.pdf",
              'pdf2':"",
              'pdf3':"",
              'question1':'How do i push codes to github?',
              'question2': "",
              'question3': ""
              }

content = preprocess_function(request_data,content_type=None)

request_data: {'pdf1': '/content/Git_Cheatsheet_KDnuggets.pdf', 'pdf2': '', 'pdf3': '', 'question1': 'How do i push codes to github?', 'question2': '', 'question3': ''}
q2: 
q3: 
['/content/Git_Cheatsheet_KDnuggets.pdf']
['How do i push codes to github?']
Text-based PDF
/tmp/pdf1
Text read from pdf: Local Changes
Add the directory to the staging area. You can add
changes from single or multiple files or even use ”.”
to add all the files to the staging area. 
$ git add <directory>
Commit the changes in the staging area with a
message “<message>” describing the changes.
$ git commit -m "<message>"
Branching
Lists all the branches in the current repository. 
$ git branch
Create a new branch <new-branch>.
$ git branch <new-branch>
Create and checkout the new branch <new-
branch>.
$ git checkout -b <new-branch>
Merge an <branch> into a current branch.
$ git merge <branch>
Delete the local branch <branch>.
$ git branch -d <branch>Update and Publish
Show the list of all locally configure remo

In [16]:
api_key = model_load_function(model_path=None)

In [27]:
response = predict_function(content, api_key)

['How do i push codes to github?']


In [28]:
res, extracted_messages = response

In [29]:
ingest_to_db = messages_to_dict(extracted_messages[0])
ingest_to_db

[{'type': 'human',
  'data': {'content': 'How do i push codes to github?',
   'additional_kwargs': {},
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'To push your code to GitHub, you can follow these steps:\n\n1. Initialize a local Git repository in your project directory (if you haven\'t already done so):\n   ```\n   git init\n   ```\n\n2. Add your files to the staging area:\n   ```\n   git add .\n   ```\n\n   This will add all the files in the current directory to the staging area. If you only want to add specific files, you can replace `.` with the file names.\n\n3. Commit your changes with a meaningful message:\n   ```\n   git commit -m "Your commit message here"\n   ```\n\n4. Add the remote repository URL of your GitHub repository:\n   ```\n   git remote add origin <remote-url>\n   ```\n\n   Replace `<remote-url>` with the URL of your GitHub repository.\n\n5. Push your changes to GitHub:\n   ```\n   git push -u origin master\n   ```\n\n   This command pushes your ch

In [30]:
retrieve_from_db = json.loads(json.dumps(ingest_to_db))
retrieve_from_db

[{'type': 'human',
  'data': {'content': 'How do i push codes to github?',
   'additional_kwargs': {},
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'To push your code to GitHub, you can follow these steps:\n\n1. Initialize a local Git repository in your project directory (if you haven\'t already done so):\n   ```\n   git init\n   ```\n\n2. Add your files to the staging area:\n   ```\n   git add .\n   ```\n\n   This will add all the files in the current directory to the staging area. If you only want to add specific files, you can replace `.` with the file names.\n\n3. Commit your changes with a meaningful message:\n   ```\n   git commit -m "Your commit message here"\n   ```\n\n4. Add the remote repository URL of your GitHub repository:\n   ```\n   git remote add origin <remote-url>\n   ```\n\n   Replace `<remote-url>` with the URL of your GitHub repository.\n\n5. Push your changes to GitHub:\n   ```\n   git push -u origin master\n   ```\n\n   This command pushes your ch

In [33]:
retrieved_messages = messages_from_dict(retrieve_from_db)
retrieved_messages

[HumanMessage(content='How do i push codes to github?', additional_kwargs={}, example=False),
 AIMessage(content='To push your code to GitHub, you can follow these steps:\n\n1. Initialize a local Git repository in your project directory (if you haven\'t already done so):\n   ```\n   git init\n   ```\n\n2. Add your files to the staging area:\n   ```\n   git add .\n   ```\n\n   This will add all the files in the current directory to the staging area. If you only want to add specific files, you can replace `.` with the file names.\n\n3. Commit your changes with a meaningful message:\n   ```\n   git commit -m "Your commit message here"\n   ```\n\n4. Add the remote repository URL of your GitHub repository:\n   ```\n   git remote add origin <remote-url>\n   ```\n\n   Replace `<remote-url>` with the URL of your GitHub repository.\n\n5. Push your changes to GitHub:\n   ```\n   git push -u origin master\n   ```\n\n   This command pushes your changes to the `master` branch of your GitHub reposit

In [31]:
out = postprocess_function(res, content_type=None)

In [32]:
print(out)

{"response": ["To push your code to GitHub, you can follow these steps:\n\n1. Initialize a local Git repository in your project directory (if you haven't already done so):\n   ```\n   git init\n   ```\n\n2. Add your files to the staging area:\n   ```\n   git add .\n   ```\n\n   This will add all the files in the current directory to the staging area. If you only want to add specific files, you can replace `.` with the file names.\n\n3. Commit your changes with a meaningful message:\n   ```\n   git commit -m \"Your commit message here\"\n   ```\n\n4. Add the remote repository URL of your GitHub repository:\n   ```\n   git remote add origin <remote-url>\n   ```\n\n   Replace `<remote-url>` with the URL of your GitHub repository.\n\n5. Push your changes to GitHub:\n   ```\n   git push -u origin master\n   ```\n\n   This command pushes your changes to the `master` branch of your GitHub repository. If you're working on a different branch, replace `master` with the name of your branch.\n\nNo

## Langchain and github seearch

In [34]:
! pip install pygithub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [35]:
import os
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents.agent_toolkits.github.toolkit import GitHubToolkit
from langchain.llms import OpenAI
from langchain.utilities.github import GitHubAPIWrapper

In [68]:
os.environ["GITHUB_REPOSITORY"] = "plaban1981/NLP-with-Python"
os.environ["GITHUB_BASE_BRANCH"] = "main"
os.environ["GITHUB_APP_ID"] = "369155"
os.environ["GITHUB_BRANCH"] = "main"
os.environ["GITHUB_APP_PRIVATE_KEY"] = "/content/langchainapp.2023-07-31.private-key.pem"

In [69]:
llm = OpenAI(temperature=0)
github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)
agent = initialize_agent(
    toolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [70]:
toolkit

GitHubToolkit(tools=[GitHubAction(name='Get Issues', description="\nThis tool will fetch a list of the repository's issues. It will return the title, and issue number of 5 issues. It takes no input.\n", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, api_wrapper=GitHubAPIWrapper(github=<github.MainClass.Github object at 0x7bbe45c8ab90>, github_repo_instance=Repository(full_name="plaban1981/NLP-with-Python"), github_repository='plaban1981/NLP-with-Python', github_app_id='369155', github_app_private_key='/content/langchainapp.2023-07-31.private-key.pem', github_branch='main', github_base_branch='main'), mode='get_issues'), GitHubAction(name='Get Issue', description='\nThis tool will fetch the title, body, and comment thread of a specific issue. **VERY IMPORTANT**: You must specify the issue number as an integer.\n', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback

In [71]:
agent.run("find the issues in the repository")



> Entering new AgentExecutor chain...
 I need to get a list of the issues
Action: Get Issues
Action Input: None
Observation: No open issues available
Thought: I now know the final answer
Final Answer: There are no open issues in the repository.

> Finished chain.


'There are no open issues in the repository.'

In [74]:
agent.run("create a new file t \n\ntest.txt\n\ntestings\in the repository")



> Entering new AgentExecutor chain...
 I need to create a file in the repository
Action: Create File
Action Input: test.txt
test contents
Observation: Unable to make file due to error:
404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/contents#get-repository-content"}
Thought: I need to specify the full file path
Action: Create File
Action Input: test/test.txt
test contents
Observation: Unable to make file due to error:
404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/contents#get-repository-content"}
Thought: I need to specify the full file path without a slash
Action: Create File
Action Input: test/test.txt
test contents
Observation: Unable to make file due to error:
404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/contents#get-repository-content"}
Thought: I need to specify the full file path without a slash and in the correct repository
Action: Create File
Action Input: <

'You need to make sure you are in the correct repository, have the correct file path, and have the correct permissions in order to create a new file in the repository.'

In [76]:
agent.run("can you check if I have the permissions to create a new file in the repository.")



> Entering new AgentExecutor chain...
 I need to check if I have the permissions to create a new file.
Action: Get Issues
Action Input: None
Observation: No open issues available
Thought: I need to check if I have the permissions to create a new file.
Action: Create File
Action Input: test/test.txt
Observation: Unable to make file due to error:
404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/contents#get-repository-content"}
Thought: I do not have the permissions to create a new file.
Final Answer: No, I do not have the permissions to create a new file in the repository.

> Finished chain.


'No, I do not have the permissions to create a new file in the repository.'